In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Flatten, Conv2D, MaxPool2D, Conv1D, MaxPool1D, GlobalAveragePooling1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import plot_model
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import re
%matplotlib inline

In [19]:
data1 = pd.read_csv("political.csv")

In [20]:
data2 = pd.read_csv("genuine.csv")

In [21]:
data1Sample = data1.sample(frac = 1)
data2Sample = data2.sample(frac = 1)

In [22]:
dataFull = data1Sample.append(data2Sample, ignore_index=True, sort=False)

In [23]:
dataFull.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148497 entries, 0 to 148496
Data columns (total 3 columns):
content    148493 non-null object
label      148497 non-null int64
count      148497 non-null int64
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [24]:
X = dataFull.content.astype(str)
Y = dataFull.label
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [25]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)
x_train = []
for each in X_train:
    x_train.append(each)
x_test = []
for each in X_test:
    x_test.append(each)

In [26]:
encoded_matrix = []
for i in range(0,len(x_train)):
    x_train[i] = x_train[i].lower()
    x_train[i] = re.sub("[^a-z]", "", x_train[i])
    sentence = x_train[0]
    encoding = np.zeros((150,26))
    for i in range(0,len(sentence)):
        encoding[i][ord(sentence[i])-97] = 1
    encoded_matrix.append(encoding)

In [27]:
matrix = np.stack(encoded_matrix)
matrix = matrix.reshape(matrix.shape[0], 150,26, 1)

MemoryError: 

In [ ]:
np.shape(matrix)

In [25]:
def ChaCNN():
    inputs = Input(name='inputs',shape=[150])
    layer = Embedding(25000, 125,input_length=150)(inputs)
    layer = LSTM(64) (layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    #layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    #layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [26]:
model = ChaCNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
plot_model(model, to_file='lstm.png', show_shapes=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 150, 125)          3125000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                48640     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_8 (Activation)    (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
Total params: 3,190,537
Trainable params: 3,190,537
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.metrics_names

['loss', 'acc']

In [39]:
model.fit(matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 100977 samples, validate on 25245 samples
Epoch 1/10
100977/100977 [==============================] - 334s 3ms/step - loss: 3.9532 - acc: 0.5568 - val_loss: 6.1856 - val_acc: 0.6120
Epoch 2/10
100977/100977 [==============================] - 330s 3ms/step - loss: 6.2543 - acc: 0.6077 - val_loss: 6.1856 - val_acc: 0.6120


In [40]:
encoded_matrix = []
for i in range(0,len(x_test)):
    x_test[i] = x_test[i].lower()
    x_test[i] = re.sub("[^a-z]", "", x_test[i])
    sentence = x_test[0]
    encoding = np.zeros((150,26))
    for i in range(0,len(sentence)):
        encoding[i][ord(sentence[i])-97] = 1
    encoded_matrix.append(encoding)

In [41]:
test_matrix = np.stack(encoded_matrix)
test_matrix = test_matrix.reshape(test_matrix.shape[0], 150,26, 1)

In [42]:
accr = model.evaluate(test_matrix,Y_test)

22275/22275 [==============================] - 29s 1ms/step


In [43]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 6.242
  Accuracy: 0.608


In [44]:
import json
import h5py

In [45]:
# serialize model to JSON
model_json = model.to_json()
with open("modelcharCNN.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("modelcharCNN.h5")
print("Saved model to disk")

Saved model to disk
